# Library Importation

In [39]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from functools import reduce
import ssl

# Constant and function definition

In [43]:
ssl._create_default_https_context = ssl._create_unverified_context
arg_read_csv = {'header':1, 'skipfooter':3, 'engine':'python'}
col_merge = ['date', 'region']
data_type_list_temperature = ['CLMTEMP', 'CLMMAXT', 'CLMMINT']
station_temperature = ['CCH','CWB','HKA','HKO','HKP','HKS','HPV','JKB','KLT','KP','KSC','KTG','LFS','NGP','PEN','PLC','SE1','SEK','SHA','SKG','SKW','SSH','SSP','STY','TC','TKL','TMS','TPO','TU1','TW','TWN','TY1','TYW','VP1','WGL','WLP','WTS','YCT']

In [40]:
def get_link_list(data):
  response = requests.get(f"https://data.gov.hk/tc-data/dataset/hk-hko-rss-{data}")
  soup = BeautifulSoup(response.text, 'html.parser')
  result = soup.find_all('a', "dataset-details__list-item-download btn btn--light")
  link_list = []
  for tag in result:
    if isinstance(link:=tag.get("href"), str) and "ALL" in link:
      link_list.append(link)
  return link_list

In [171]:
def digit_or_null(value):
    try:
        float(value)
    except:
        return np.nan
    else:
        return float(value)

In [186]:
def read_csv(link, year=0):
  df = pd.read_csv(link, **arg_read_csv)
  col_name = df.columns[0]
  df_temp = df.reset_index().iloc[1:].dropna(subset='level_3')
  df_rename = df_temp.rename(columns={'level_0': 'year', 'level_1': 'month', 'level_2': 'day', 'level_3': col_name, col_name:'completeness'})
  df_valid_data = df_rename[(df_rename['year'] != '1900') | (df_rename['month'] != '2') | (df_rename['day'] != '29')]
  df_valid_data = df_valid_data.copy()
  df_valid_data['date'] = pd.to_datetime(df_valid_data[['year','month','day']])
  df_date = df_valid_data.set_index('date')
  df_date_filter = df_date[df_date.index.year >= year]
  df_final = df_date_filter[col_name].apply(lambda x: digit_or_null(x))
  return df_final

In [215]:
def temperature(data_type, station, year=0):
  df = pd.read_csv(f"https://data.weather.gov.hk/weatherAPI/opendata/opendata.php?dataType={data_type}&rformat=csv&station={station}",
            engine='python',
            header=1,
            skipfooter=3,
            sep=None)
  col_name = df.columns[0]
  df_temp = df.iloc[1:,0].str.split(",", expand=True)
  df_rename = df_temp.rename(columns={0: 'year', 1: 'month', 2: 'day', 3: col_name, 4:'completeness'})
  df_valid_data = df_rename[(df_rename['year'] != '1900') | (df_rename['month'] != '2') | (df_rename['day'] != '29')].reset_index()
  df_valid_data = df_valid_data.copy()
  df_valid_data['date'] = pd.to_datetime(df_valid_data[['year','month','day']])
  df_date = df_valid_data.set_index('date')
  df_date_filter = df_date[df_date.index.year >= year]
  df_final = df_date_filter[col_name].apply(lambda x: digit_or_null(x))
  return df_final

In [72]:
def melt(df):
  if isinstance(df, pd.Series):
    df = df.to_frame()
  df_melt = pd.melt(df.reset_index(), id_vars=['date'])
  df_melt['region'] = df_melt['variable'].apply(lambda x: x.split("- ")[1] if "-" in x else x.split("at the ")[1])
  df_melt['type'] = df_melt['variable'].apply(lambda x: x.split(" -")[0] if "-" in x else x.split(" at the")[0].split("Daily ")[1])
  col_name = df_melt['type'].value_counts().index[0]
  return df_melt[['date', 'region', 'value']].rename(columns={'value':col_name})

In [48]:
def merge(list_df): 
    return reduce(lambda  left, right: pd.merge(left, right, on=col_merge, how='outer'), list_df)

In [126]:
def melt_merge(data, year=0):   
    return merge([melt(read_csv(link_csv,year)) for link_csv in get_link_list(data)])

In [127]:
def concat_melt(data,year=0):
    return melt(pd.concat([read_csv(link,year) for link in get_link_list(data)], axis=1))

# Daily Maximum Mean Heat Index

In [187]:
daily_maximum_mean_heat_index = melt_merge("daily-maximum-mean-heat-index", 2014)

In [188]:
daily_maximum_mean_heat_index

,date,region,Mean HKHI,Max HKHI
0,2014-05-30,King's Park,28.3,30.1
1,2014-05-31,King's Park,28.6,30.3
2,2014-06-01,King's Park,29.0,30.2
3,2014-06-02,King's Park,28.5,29.8
4,2014-06-03,King's Park,27.9,30.1
...,...,...,...,...
3803,2024-10-27,King's Park,24.6,25.9
3804,2024-10-28,King's Park,21.8,23.3
3805,2024-10-29,King's Park,21.9,23.9
3806,2024-10-30,King's Park,23.2,25.4


# Daily Mean Amount Of Cloud

In [189]:
daily_mean_amount_of_cloud = melt_merge("daily-mean-amount-of-cloud", 2014)

In [190]:
daily_mean_amount_of_cloud

,date,region,Mean Amount of Cloud (%)
0,2014-01-01,Hong Kong Observatory,0.0
1,2014-01-02,Hong Kong Observatory,49.0
2,2014-01-03,Hong Kong Observatory,28.0
3,2014-01-04,Hong Kong Observatory,0.0
4,2014-01-05,Hong Kong Observatory,2.0
...,...,...,...
3982,2024-11-26,Hong Kong Observatory,45.0
3983,2024-11-27,Hong Kong Observatory,45.0
3984,2024-11-28,Hong Kong Observatory,40.0
3985,2024-11-29,Hong Kong Observatory,32.0


# Daily Mean Pressure

In [191]:
daily_mean_pressure = concat_melt("daily-mean-pressure", 2014)

In [192]:
daily_mean_pressure

,date,region,Mean Pressure (hPa)
0,2014-01-01,Hong Kong International Airport,1019.3
1,2014-01-02,Hong Kong International Airport,1016.6
2,2014-01-03,Hong Kong International Airport,1015.9
3,2014-01-04,Hong Kong International Airport,1018.0
4,2014-01-05,Hong Kong International Airport,1018.5
...,...,...,...
47839,2024-11-26,Wetland Park,1019.6
47840,2024-11-27,Wetland Park,1021.3
47841,2024-11-28,Wetland Park,1022.2
47842,2024-11-29,Wetland Park,1021.0


# Daily Total Rainfall

In [193]:
daily_total_rainfall = concat_melt("daily-total-rainfall", 2014)

In [194]:
daily_total_rainfall

,date,region,Total Rainfall (mm)
0,2014-01-01,Hong Kong International Airport,0.0
1,2014-01-02,Hong Kong International Airport,0.0
2,2014-01-03,Hong Kong International Airport,0.0
3,2014-01-04,Hong Kong International Airport,0.0
4,2014-01-05,Hong Kong International Airport,0.0
...,...,...,...
99670,2024-11-26,Wetland Park,2.0
99671,2024-11-27,Wetland Park,0.0
99672,2024-11-28,Wetland Park,0.0
99673,2024-11-29,Wetland Park,0.0


# Daily Mean Relative Humidity

In [208]:
daily_mean_relative_humidity = concat_melt("daily-mean-relative-humidity", 2014)

In [209]:
# Missed New Tsing Yi Station
daily_mean_relative_humidity

,date,region,Mean Relative Humidity (%)
0,2014-01-01,Hong Kong International Airport,45.0
1,2014-01-02,Hong Kong International Airport,67.0
2,2014-01-03,Hong Kong International Airport,63.0
3,2014-01-04,Hong Kong International Airport,46.0
4,2014-01-05,Hong Kong International Airport,48.0
...,...,...,...
95683,2024-11-26,Wong Chuk Hang,60.0
95684,2024-11-27,Wong Chuk Hang,44.0
95685,2024-11-28,Wong Chuk Hang,33.0
95686,2024-11-29,Wong Chuk Hang,36.0


# Daily Maximum, Mean and Minimum Temperatures

In [222]:
daily_temperature = merge([melt(pd.concat([temperature(data_type, station, 2014) for station in station_temperature], axis=1)) for data_type in data_type_list_temperature])

In [223]:
daily_temperature

,date,region,Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C)
0,2014-01-01,Cheung Chau,15.7,20.6,12.3
1,2014-01-01,Clear Water Bay,NaN,NaN,NaN
2,2014-01-01,Happy Valley,13.9,21.0,8.7
3,2014-01-01,Hong Kong International Airport,16.1,19.5,11.5
4,2014-01-01,Hong Kong Observatory,15.7,18.9,13.0
...,...,...,...,...,...
151501,2024-11-30,Tuen Mun Children and Juvenile Home,18.1,23.7,14.7
151502,2024-11-30,Waglan Island,19.3,23.4,16.9
151503,2024-11-30,Wetland Park,16.9,24.2,11.6
151504,2024-11-30,Wong Chuk Hang,17.2,23.2,13.1


# Daily Global Solar Radiation

In [232]:
daily_global_solar_radiation = concat_melt("daily-global-solar-radiation", 2014)

In [233]:
daily_global_solar_radiation

,date,region,Global Solar Radiation (MJ/m&sup2;)
0,2014-01-01,King's Park,16.11
1,2014-01-02,King's Park,11.77
2,2014-01-03,King's Park,13.92
3,2014-01-04,King's Park,13.49
4,2014-01-05,King's Park,14.17
...,...,...,...
7969,2024-11-26,Kau Sai Chau,17.80
7970,2024-11-27,Kau Sai Chau,17.41
7971,2024-11-28,Kau Sai Chau,18.00
7972,2024-11-29,Kau Sai Chau,18.22


# Daily Maximum Mean UV Index

In [248]:
daily_maximum_mean_uv_index = melt_merge("daily-maximum-mean-uv-index", 2014)

In [249]:
daily_maximum_mean_uv_index

,date,region,Mean UV Indices(7 a.m. to 6 p.m.),Max UV Indices(15-minute average)
0,2014-01-01,King's Park,2.0,5.0
1,2014-01-02,King's Park,2.0,5.0
2,2014-01-03,King's Park,2.0,4.0
3,2014-01-04,King's Park,2.0,4.0
4,2014-01-05,King's Park,2.0,4.0
...,...,...,...,...
3982,2024-11-26,King's Park,3.0,6.0
3983,2024-11-27,King's Park,2.0,5.0
3984,2024-11-28,King's Park,3.0,6.0
3985,2024-11-29,King's Park,3.0,6.0


# Daily Total Bright Sunshine

In [253]:
daily_total_bright_sunshine = melt_merge("daily-total-bright-sunshine", 2014)

In [254]:
daily_total_bright_sunshine

,date,region,Total Bright Sunshine (hours)
0,2014-01-01,King's Park,9.5
1,2014-01-02,King's Park,4.5
2,2014-01-03,King's Park,8.6
3,2014-01-04,King's Park,8.8
4,2014-01-05,King's Park,9.0
...,...,...,...
3982,2024-11-26,King's Park,9.8
3983,2024-11-27,King's Park,9.8
3984,2024-11-28,King's Park,9.8
3985,2024-11-29,King's Park,9.8


# Combine All

In [270]:
data_list = [daily_maximum_mean_heat_index, daily_mean_amount_of_cloud, daily_mean_pressure, daily_total_rainfall, daily_mean_relative_humidity, daily_temperature, daily_global_solar_radiation, daily_maximum_mean_uv_index ,daily_total_bright_sunshine]

In [272]:
data = merge(data_list)

In [273]:
data

,date,region,Mean HKHI,Max HKHI,Mean Amount of Cloud (%),Mean Pressure (hPa),Total Rainfall (mm),Mean Relative Humidity (%),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Global Solar Radiation (MJ/m&sup2;),Mean UV Indices(7 a.m. to 6 p.m.),Max UV Indices(15-minute average),Total Bright Sunshine (hours)
0,2014-01-01,Cheung Chau,NaN,NaN,NaN,1019.0,0.0,50.0,15.7,20.6,12.3,NaN,NaN,NaN,NaN
1,2014-01-01,Ching Pak House(Tsing Yi),NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-01-01,Clear Water Bay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-01-01,Happy Valley,NaN,NaN,NaN,NaN,NaN,NaN,13.9,21.0,8.7,NaN,NaN,NaN,NaN
4,2014-01-01,Hong Kong International Airport,NaN,NaN,NaN,NaN,NaN,NaN,16.1,19.5,11.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163462,2024-11-30,Tuen Mun Children and Juvenile Home,NaN,NaN,NaN,NaN,0.0,47.0,18.1,23.7,14.7,NaN,NaN,NaN,NaN
163463,2024-11-30,Waglan Island,NaN,NaN,NaN,1017.7,0.0,57.0,19.3,23.4,16.9,NaN,NaN,NaN,NaN
163464,2024-11-30,Wetland Park,NaN,NaN,NaN,1017.6,0.0,65.0,16.9,24.2,11.6,NaN,NaN,NaN,NaN
163465,2024-11-30,Wong Chuk Hang,NaN,NaN,NaN,NaN,NaN,62.0,17.2,23.2,13.1,NaN,NaN,NaN,NaN


In [279]:
data.value_counts('region')

region
Cheung Chau                            3987
Ching Pak House(Tsing Yi)              3987
Clear Water Bay                        3987
Happy Valley                           3987
Hong Kong International Airport        3987
Hong Kong International Airport        3987
Hong Kong Observatory                  3987
Hong Kong Park                         3987
Kai Tak                                3987
Kai Tak Runway Park                    3987
Kau Sai Chau                           3987
King's Park                            3987
Kowloon City                           3987
Kwun Tong                              3987
Lau Fau Shan                           3987
New Tsing Yi Station                   3987
Ngong Ping                             3987
Pak Tam Chung (Tsak Yue Wu)            3987
Peng Chau                              3987
Sai Kung                               3987
Sha Tin                                3987
Sham Shui Po                           3987
Shau Kei Wan             